In [1]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import rioxarray

data_dir = Path('../data')

In [2]:
metrics_dir = data_dir / 'outputs/plots/metrics/x1-y1-z1/net_cdf'
plot_summary_dir = data_dir / 'outputs/plots/metrics'

def read_plot_metrics(plot_id: str):
    metrics = xr.open_dataset(metrics_dir / f"{plot_id}_with_cover.nc", decode_coords='all')
    metrics.load()
    metrics.close()
    return metrics

In [3]:
plots = gpd.read_file(data_dir / "outputs/plots/plots.geojson")
plots = plots.set_index('id')
plots

,site,plot_number,site_plot_id,geometry
id,,,,
AGG_O_01_P1,AGG_O_01,1,AGG_O_01_P1,"POLYGON ((463042.83 5259846.736, 463025.797 52..."
AGG_O_01_P2,AGG_O_01,2,AGG_O_01_P2,"POLYGON ((463124.556 5259819.234, 463116.068 5..."
AGG_O_01_P3,AGG_O_01,3,AGG_O_01_P3,"POLYGON ((463201.174 5259815.806, 463200.551 5..."
AGG_O_01_P4,AGG_O_01,4,AGG_O_01_P4,"POLYGON ((463257.777 5259801.962, 463245.303 5..."
AGG_O_01_P5,AGG_O_01,5,AGG_O_01_P5,"POLYGON ((463303.022 5259789.552, 463289.794 5..."
...,...,...,...,...
ULY_Y_96_P1,ULY_Y_96,1,ULY_Y_96_P1,"POLYGON ((455348.476 5275834.517, 455348.154 5..."
ULY_Y_96_P2,ULY_Y_96,2,ULY_Y_96_P2,"POLYGON ((455296.576 5275837.224, 455299.223 5..."
ULY_Y_96_P3,ULY_Y_96,3,ULY_Y_96_P3,"POLYGON ((455252.092 5275834.16, 455248.849 52..."


In [ ]:
def create_plot_summary(row: gpd.GeoSeries) -> pd.Series:
    id = row.name
    metrics = read_plot_metrics(id)

    mean_metrics_names = [
        "point_density",
        "pulse_density",
        "scan_angle_mean",
        "chm",
        "mean_height",
        "median_height",
        "q10_veg_height",
        "q20_veg_height",
        "q30_veg_height",
        "q40_veg_height",
        "q50_veg_height",
        "q60_veg_height",
        "q70_veg_height",
        "q80_veg_height",
        "q90_veg_height",
        "crr",
        "sd_veg_height",
        'cv_veg_height',
        'skew_veg_height',
        'kurt_veg_height',
        'gini_veg_height',
        'canopy_cover_1m',
        'fhd',
        'vci',
        'shann_capture',
        'shann_capture',
    ]
    # Skip point and pulse density and scan angle
    sd_metric_names = mean_metrics_names[3:]
    # CV for all the ones that are in height units
    cv_metric_names = [
        "chm",
        "mean_height",
        "median_height",
        "q10_veg_height",
        "q20_veg_height",
        "q30_veg_height",
        "q40_veg_height",
        "q50_veg_height",
        "q60_veg_height",
        "q70_veg_height",
        "q80_veg_height",
        "q90_veg_height",
    ]
    
    # done separately as they may not exist
    cover_metrics = [
        'groundstorey_capture',
        'understorey_capture',
        'midstorey_capture',
        'upperstorey_capture'
    ]

    mean_metrics : pd.Series = metrics[mean_metrics_names].mean(dim=['x', 'y']).to_pandas()
    sd_metrics : pd.Series = metrics[mean_metrics_names].sd(dim=['x', 'y']).to_pandas()
    sd_metrics = sd_metrics.add_suffix('__sd')

    return mean_metrics

create_plot_summary(plots.iloc[0])

point_density      94.731332
pulse_density      48.682927
scan_angle_mean    -0.070185
chm                 9.080404
mean_height         4.064385
median_height       4.576714
q10_veg_height      0.827366
q20_veg_height      2.094856
q30_veg_height      3.608292
q40_veg_height      4.965329
q50_veg_height      5.942358
q60_veg_height      6.597463
q70_veg_height      7.135069
q80_veg_height      7.627270
q90_veg_height      8.156483
crr                 0.420128
sd_veg_height       2.901770
cv_veg_height       0.629295
skew_veg_height    -0.488462
kurt_veg_height    -0.295746
gini_veg_height     0.326379
canopy_cover_1m     0.563681
fhd                 1.716545
vci                 0.593884
shann_capture       1.544037
dtype: float64

In [34]:
metrics = read_plot_metrics('AGG_O_01_P1')
metrics

<xarray.Dataset> Size: 8MB
Dimensions:                     (y: 65, x: 67, z: 18)
Coordinates:
  * x                           (x) float64 536B 4.63e+05 4.63e+05 ... 4.63e+05
  * y                           (y) float64 520B 5.26e+06 5.26e+06 ... 5.26e+06
    spatial_ref                 int64 8B 0
  * z                           (z) float64 144B 0.0 1.0 2.0 ... 15.0 16.0 17.0
Data variables: (12/53)
    point_density               (y, x) float64 35kB nan nan nan ... nan nan nan
    pulse                       (y, x) float64 35kB nan nan nan ... nan nan nan
    chm                         (y, x) float64 35kB nan nan nan ... nan nan nan
    mean_height                 (y, x) float64 35kB nan nan nan ... nan nan nan
    median_height               (y, x) float64 35kB nan nan nan ... nan nan nan
    q10_veg_height              (y, x) float64 35kB nan nan nan ... nan nan nan
    ...                          ...
    understorey_capture         (y, x) float64 35kB nan nan nan ... nan nan nan
    understorey_capture_w       (y, x) float64 35kB nan nan nan ... nan nan nan
    upperstorey_rel_density     (y, x) float64 35kB nan nan nan ... nan nan nan
    upperstorey_rel_density_w   (y, x) float64 35kB nan nan nan ... nan nan nan
    upperstorey_capture         (y, x) float64 35kB nan nan nan ... nan nan nan
    upperstorey_capture_w       (y, x) float64 35kB nan nan nan ... nan nan nan
Attributes:
    xy_bin_size:          1
    z_bin_size:           1
    groundstorey_bounds:  (0, 1)
    understorey_bounds:   (1, 4)
    upperstorey_bounds:   (4, nan)